In [1]:
import torchreid
import numpy as np
import sklearn.model_selection
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import glob
import os.path as osp

from torchreid.data import ImageDataset
torchreid.models.show_avai_models()

c:\Users\User\anaconda3\envs\torchreid2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [2]:
import torch

torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.get_device_name(0)
torch. __version__

'1.12.1+cu113'

In [ ]:
#Code for visualizing feature extraction for osnet model
# python tools/visualize_actmap.py --root reid-data -d market1501 -m osnet_x1_0 --weights log/osnet_x1_0/model/model.pth.tar-60 --save-dir log/visactmap_osnet_x1_0_market1501_2

In [5]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)




Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset


c:\Users\User\Desktop\fyp project kaiyang\deep-person-reid\torchreid\data\datasets\image\market1501.py:37: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  warnings.warn(


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [17]:
datamanager = torchreid.data.VideoDataManager(
        root='reid-data',
        sources='mars',
        height=256,
        width=128,
        batch_size_train=3,
        batch_size_test=3,
        seq_len=15,
        sample_method='evenly'
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Mars
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   625 |        8298 |         6
  query    |   626 |        1980 |         6
  gallery  |   621 |        9330 |         6
  -------------------------------------------
=> Loading test (target) dataset
=> Loaded Mars
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   625 |        8298 |         6
  query    |   626 |        1980 |         6
  gallery  |   621 |        9330 

In [18]:
#This is for video based reid model
model = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Successfully loaded imagenet pretrained weights from "C:\Users\User/.cache\torch\checkpoints\osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [7]:
model = torchreid.models.build_model(
    name="resnet101",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [8]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [19]:
#This is for video based person reid
engine = torchreid.engine.VideoSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler,
    pooling_method='avg'
)

In [7]:
#This is for video based person reid
engine.run(
    max_epoch=60,
    save_dir='log/resnet_testing_1852023',
    print_freq=10
)

=> Start training


In [8]:
weight_path = "log/resnet50new/model/model.pth.tar-60"
torchreid.utils.load_pretrained_weights(model, weight_path)



Successfully loaded pretrained weights from "log/resnet50new/model/model.pth.tar-60"


In [9]:
engine.run(
    save_dir="log/resnet101_270523",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=True,
    visrank=True
)

##### Evaluating market1501 (source) #####
Extracting features from query set ...


KeyboardInterrupt: 

In [10]:
engine.run(
    save_dir="log/resnet101_270523",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training
epoch: [1/60][10/404]	time 0.197 (0.919)	data 0.000 (0.633)	eta 6:11:06	loss 6.9331 (6.7462)	acc 3.1250 (0.9375)	lr 0.000300
epoch: [1/60][20/404]	time 0.191 (0.556)	data 0.000 (0.316)	eta 3:44:33	loss 6.8554 (6.7878)	acc 0.0000 (1.4062)	lr 0.000300
epoch: [1/60][30/404]	time 0.185 (0.434)	data 0.000 (0.211)	eta 2:55:03	loss 6.7671 (6.7620)	acc 0.0000 (1.1458)	lr 0.000300
epoch: [1/60][40/404]	time 0.186 (0.372)	data 0.000 (0.158)	eta 2:29:58	loss 6.7014 (6.7252)	acc 3.1250 (1.0156)	lr 0.000300
epoch: [1/60][50/404]	time 0.186 (0.335)	data 0.001 (0.127)	eta 2:14:54	loss 6.5790 (6.6949)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/60][60/404]	time 0.185 (0.310)	data 0.001 (0.106)	eta 2:04:50	loss 6.5487 (6.6765)	acc 0.0000 (0.7812)	lr 0.000300
epoch: [1/60][70/404]	time 0.186 (0.292)	data 0.000 (0.091)	eta 1:57:40	loss 6.5299 (6.6577)	acc 0.0000 (0.8036)	lr 0.000300
epoch: [1/60][80/404]	time 0.186 (0.279)	data 0.000 (0.079)	eta 1:52:15	loss 6.6020 (6.6444)	acc 0.0000 (0.

In [ ]:
# from torchreid.utils import FeatureExtractor

# extractor = FeatureExtractor(
#     model_name='resnet50',
#     model_path='log/resnet50/model/model.pth.tar-60',
#     device='cuda'
# )

# image_list = [
#     'reid-data/market1501/bounding_box_train/0002_c1s1_000776_01.jpg',
#     'reid-data/market1501/bounding_box_train/0002_c1s1_000801_01.jpg'
# ]

# features = extractor(image_list)
# print(features.shape) # output (5, 512)

In [ ]:
# python tools/visualize_actmap.py --root reid-data/market1501 -d market1501 -m resnet50 --weights log/resnet50/model/model.pth.tar-60 --save-dir log/visactmap_resnet50_market1501

In [ ]:
import os
os.getcwd() 

In [ ]:
tensorboard --logdir log/osnet_x1_0/events.out.tfevents.1669385751.DESKTOP-LFK44LR.2292.1

In [15]:
weight_path = "log/osnet_x1_0/model/model.pth.tar-60"
torchreid.utils.load_pretrained_weights(model, weight_path)

Successfully loaded pretrained weights from "log/osnet_x1_0/model/model.pth.tar-60"


In [16]:

engine.run(
    save_dir="log/osnet_x1_0",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=True,
    visrank=True
)

##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Speed: 0.0337 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 76.5%
CMC curve
Rank-1  : 91.8%
Rank-5  : 96.6%
Rank-10 : 97.7%
Rank-20 : 98.5%
# query: 3368
# gallery 15913
Visualizing top-10 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3

In [ ]:
weight_path = "log/osnet_x1_0-softmax-mars/model/model.pth.tar-60"
torchreid.utils.load_pretrained_weights(model, weight_path)

### Running ResNet50 and OSNet models on Occluded-ReID

In [21]:
# from __future__ import absolute_import
# from __future__ import print_function
# from __future__ import division

# import glob
# import os.path as osp

# from torchreid.data import ImageDataset

# class OccludedREID(ImageDataset):
#     dataset_dir = 'occluded_reid'

#     def __init__(self, root='', **kwargs):
#         self.root = osp.abspath(osp.expanduser(root))
#         self.occluded_dir = osp.join(self.root, self.dataset_dir, 'occluded_body_images')
#         self.whole_dir = osp.join(self.root, self.dataset_dir, 'whole_body_images')

#         train = self._process_dir(self.whole_dir, relabel=True)
#         query = self._process_dir(self.whole_dir, relabel=False)
#         gallery = self._process_dir(self.occluded_dir, relabel=False)

#         super(OccludedREID, self).__init__(train, query, gallery, **kwargs)

#     def _process_dir(self, dir_path, relabel=False):
#         img_paths = glob.glob(osp.join(dir_path, '*', '*.tif'))

#         pid_container = set()
#         for img_path in img_paths:
#             pid = int(osp.basename(osp.dirname(img_path)))
#             pid_container.add(pid)

#         pid2label = {pid: label for label, pid in enumerate(pid_container)}

#         dataset = []
#         for img_path in img_paths:
#             pid = int(osp.basename(osp.dirname(img_path)))
#             if relabel: pid = pid2label[pid]
#             img_name = osp.basename(img_path)
#             camid = int(img_name.split('_')[1].split('.')[0])
#             dataset.append((img_path, pid, camid))

#         return dataset

from torchreid.data import ImageDataset
from sklearn.model_selection import train_test_split

class OccludedREID(ImageDataset):
    dataset_dir = 'occluded_reid'

    def __init__(self, root='', split_ratio=0.5, **kwargs):
        self.root = osp.abspath(osp.expanduser(root))
        self.occluded_dir = osp.join(self.root, self.dataset_dir, 'occluded_body_images')
        self.whole_dir = osp.join(self.root, self.dataset_dir, 'whole_body_images')

        pid_container = self._get_pid_container(self.whole_dir)
        train_pids, test_pids = train_test_split(list(pid_container), test_size=split_ratio, random_state=42)

        train = self._process_dir(self.whole_dir, train_pids, relabel=True)
        gallery = self._process_dir(self.whole_dir, test_pids, relabel=False)
        query = self._process_dir(self.occluded_dir, test_pids, relabel=False)

        super(OccludedREID, self).__init__(train, query, gallery, **kwargs)

    def _get_pid_container(self, dir_path):
        img_paths = glob.glob(osp.join(dir_path, '*', '*.tif'))

        pid_container = set()
        for img_path in img_paths:
            pid = int(osp.basename(osp.dirname(img_path)))
            pid_container.add(pid)

        return pid_container

    def _process_dir(self, dir_path, pids, relabel=False):
        img_paths = glob.glob(osp.join(dir_path, '*', '*.tif'))

        pid2label = {pid: label for label, pid in enumerate(pids)}

        dataset = []
        for img_path in img_paths:
            pid = int(osp.basename(osp.dirname(img_path)))
            if pid not in pids: continue
            if relabel: pid = pid2label[pid]
            img_name = osp.basename(img_path)
            camid = int(img_name.split('_')[1].split('.')[0])
            dataset.append((img_path, pid, camid))

        return dataset

# Register your dataset
import torchreid
torchreid.data.register_image_dataset('occluded_reid', OccludedREID)

In [22]:
# # This is for softmax loss
# datamanager = torchreid.data.ImageDataManager(
#     root='reid-data',
#     sources='occluded_reid',
#     height=256,
#     width=128,
#     batch_size_train=32,
#     batch_size_test=100,
#     workers=0, 
# )

#This is for Triplet loss
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='occluded_reid',
    height=240,
    width=240,
    combineall=False,
    batch_size_train=20, #setting this at 64 crashes the run. IDK why lol
    batch_size_test=20,
    num_instances=4,
    workers=0,
    train_sampler='RandomIdentitySampler' # this is important
)

Building train transforms ...
+ resize to 240x240
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 240x240
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded OccludedREID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   100 |      500 |         5
  query    |   100 |      500 |         5
  gallery  |   100 |      500 |         5
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded OccludedREID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   100 |      500 |         5
  query    |   100 |      500 |         5
  gallery  |   100 |      500 |         5
  ---------

## Testing playground for Model Improvements

### Model on Softmax engine Occluded-Reid dataset


In [6]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir="log/resnet50_market1501triplet_papersetting_090623",
    max_epoch=100,
    eval_freq=10,
    print_freq=10,
    test_only=False,
    normalize_feature= True
)

=> Start training
epoch: [1/100][10/20]	time 0.200 (0.207)	data 0.062 (0.048)	eta 0:06:51	loss 5.6447 (5.1113)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [1/100][20/20]	time 0.170 (0.194)	data 0.031 (0.044)	eta 0:06:23	loss 5.1854 (5.1997)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [2/100][10/20]	time 0.170 (0.172)	data 0.032 (0.032)	eta 0:05:38	loss 4.5846 (4.6798)	acc 0.0000 (1.5000)	lr 0.000300
epoch: [2/100][20/20]	time 0.168 (0.170)	data 0.033 (0.032)	eta 0:05:33	loss 4.6459 (4.6521)	acc 0.0000 (0.7500)	lr 0.000300
epoch: [3/100][10/20]	time 0.167 (0.171)	data 0.030 (0.032)	eta 0:05:32	loss 4.6791 (4.6333)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [3/100][20/20]	time 0.170 (0.171)	data 0.032 (0.032)	eta 0:05:30	loss 4.6226 (4.6443)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [4/100][10/20]	time 0.166 (0.168)	data 0.030 (0.031)	eta 0:05:24	loss 4.5493 (4.5558)	acc 0.0000 (1.5000)	lr 0.000300
epoch: [4/100][20/20]	time 0.168 (0.169)	data 0.030 (0.031)	eta 0:05:23	loss 4.6649 (4.6113)	acc 0.0000 (0.

### Model on Triplet loss engine Occluded-reid dataset

In [39]:
model = torchreid.models.build_model(
    name='resnet50mid',
    num_classes=datamanager.num_train_pids,
    loss='triplet'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003,
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.ImageTripletEngine(
    datamanager, model, optimizer, margin=0.3,
    weight_t=0.7, weight_x=1, scheduler=scheduler
)
engine.run(
    max_epoch=100,
    save_dir='log/resnet50mid_forgettingbest',
    normalize_feature= True
)

=> Start training
epoch: [1/100][10/20]	time 0.163 (0.187)	data 0.033 (0.033)	eta 0:06:11	loss_t 1.1649 (1.2456)	loss_x 6.2809 (5.4549)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [1/100][20/20]	time 0.155 (0.173)	data 0.033 (0.034)	eta 0:05:41	loss_t 2.5821 (1.0970)	loss_x 6.7853 (6.0673)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [2/100][10/20]	time 0.160 (0.159)	data 0.036 (0.037)	eta 0:05:13	loss_t 1.2395 (1.0717)	loss_x 4.1377 (4.1289)	acc 0.0000 (19.0000)	lr 0.000300
epoch: [2/100][20/20]	time 0.160 (0.159)	data 0.034 (0.035)	eta 0:05:10	loss_t 0.3822 (1.0313)	loss_x 4.5995 (4.2333)	acc 0.0000 (10.5000)	lr 0.000300
epoch: [3/100][10/20]	time 0.160 (0.159)	data 0.032 (0.035)	eta 0:05:09	loss_t 0.1391 (0.4736)	loss_x 3.8352 (3.5204)	acc 0.0000 (28.5000)	lr 0.000300
epoch: [3/100][20/20]	time 0.161 (0.159)	data 0.036 (0.035)	eta 0:05:08	loss_t 0.4365 (0.3962)	loss_x 4.3307 (3.7025)	acc 0.0000 (16.5000)	lr 0.000300
epoch: [4/100][10/20]	time 0.157 (0.159)	data 0.033 (0.033)	eta 0:05:06	loss_t

### Model on Softmax Market-1501 dataset

In [3]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)


model = torchreid.models.build_model(
    name="osnet_x0_75",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir="log/osnet_x0_75_market1501softmax_normalized_310523",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=False,
    normalize_feature= True
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset


c:\Users\User\Desktop\fyp project kaiyang\deep-person-reid\torchreid\data\datasets\image\market1501.py:37: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  warnings.warn(


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************


Successfully loaded imagenet pretrained weights from "C:\Users\

: 

: 

### Model on Triplet Market-1501 dataset

In [40]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

model = torchreid.models.build_model(
    name='osnet_x0_75',
    num_classes=datamanager.num_train_pids,
    loss='triplet'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.ImageTripletEngine(
    datamanager, model, optimizer, margin=0.3,
    weight_t=0.7, weight_x=1, scheduler=scheduler
)
engine.run(
    max_epoch=60,
    save_dir='log/osnet_x0_75_market1501triplet_normalized_310523',
    print_freq=10,
    normalize_feature= True
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset


c:\Users\User\Desktop\fyp project kaiyang\deep-person-reid\torchreid\data\datasets\image\market1501.py:37: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  warnings.warn(


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




### Ensemble Deep Learning on the Market-1501 dataset

#### Two OSNet models combined

In [44]:
from torchreid.models import build_model
import torch.nn as nn
from torchreid.utils import FeatureExtractor
from torch.nn import functional as F
import numpy as np
from torchreid import metrics

# Extract features from the testing data
test_loaders = datamanager.test_loader

query_loader = test_loaders['market1501']['query']
gallery_loader = test_loaders['market1501']['gallery']


#Define extractor
extractor1 = FeatureExtractor(model_path='log/osnet_x1_0_market1501final/model/model.pth.tar-100', model_name='osnet_x1_0', device='cuda')
use_gpu = torch.cuda.is_available()

extractor2  = FeatureExtractor(model_path='log/osnet_x0_75_market1501final/model/model.pth.tar-100', model_name='osnet_x0_75', device='cuda')

@torch.no_grad()
def feature_extraction(data_loader, extractor, use_gpu):
    f_, pids_, camids_ = [], [], []
    for batch_idx, data in enumerate(data_loader):
        imgs = data['img']
        pids = data['pid']
        camids = data['camid']
        if use_gpu:
            imgs = imgs.cuda()
        features = extractor(imgs)  # use the extractor instance directly
        features = features.cpu()
        f_.append(features)
        pids_.extend(pids.tolist())
        camids_.extend(camids.tolist())
    f_ = torch.cat(f_, 0)
    pids_ = np.asarray(pids_)
    camids_ = np.asarray(camids_)
    return f_, pids_, camids_

#Extracting features using extractor1
print('Extracting features from query set ...')
qf, q_pids, q_camids = feature_extraction(query_loader, extractor1, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(qf.size(0), qf.size(1)))

print('Extracting features from gallery set ...')
gf, g_pids, g_camids = feature_extraction(gallery_loader, extractor1, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(gf.size(0), gf.size(1)))

#Extracting features using extractor2
print('Extracting features from query set ...')
qf2, q_pids2, q_camids2 = feature_extraction(query_loader, extractor2, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(qf.size(0), qf.size(1)))

print('Extracting features from gallery set ...')
gf2, g_pids2, g_camids2 = feature_extraction(gallery_loader, extractor2, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(gf.size(0), gf.size(1)))

qf_avg = (qf + qf2  ) / 2
gf_avg = (gf + gf2 ) / 2

#Compute distance matrix for extractor1
print('Computing distance matrix with metric={} ...'.format("euclidean"))
distmat = metrics.compute_distance_matrix(qf_avg, gf_avg, metric="euclidean")
distmat = distmat.numpy()



print('Computing CMC and mAP ...')
cmc, mAP = metrics.evaluate_rank(
    distmat,
    q_pids,
    g_pids,
    q_camids,
    g_camids,
)

# print results
print(f"mAP: {mAP:.1%}, Rank-1: {cmc[0]:.1%}, Rank-5: {cmc[4]:.1%}, Rank-10: {cmc[9]:.1%},Rank-20: {cmc[19]:.1%}")

Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352
Successfully loaded pretrained weights from "log/osnet_x1_0_market1501final/model/model.pth.tar-100"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x0_75
- params: 1,299,224
- flops: 571,754,536
Successfully loaded pretrained weights from "log/osnet_x0_75_market1501final/model/model.pth.tar-100"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Extracting features from query set ...
Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Extracting features from query set ...
Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
mAP: 80.6%, Rank-1: 93.1%, Rank-5: 97.5%, Rank-10: 98.5%,Rank-20: 99.1

### Ensemble Learning On Occluded Reid

In [38]:
from torchreid.models import build_model
import torch.nn as nn
from torchreid.utils import FeatureExtractor
from torch.nn import functional as F
import numpy as np
from torchreid import metrics

# Extract features from the testing data
test_loaders = datamanager.test_loader

query_loader = test_loaders['occluded_reid']['query']
gallery_loader = test_loaders['occluded_reid']['gallery']


#Define extractor
extractor1 = FeatureExtractor(model_path='log/densenet169_ocreidfinal/model/model.pth.tar-100', model_name='densenet169', device='cuda')
use_gpu = torch.cuda.is_available()

extractor2  = FeatureExtractor(model_path='log/densenet161_ocreidfinal/model/model.pth.tar-100', model_name='densenet161', device='cuda')

@torch.no_grad()
def feature_extraction(data_loader, extractor, use_gpu):
    f_, pids_, camids_ = [], [], []
    for batch_idx, data in enumerate(data_loader):
        imgs = data['img']
        pids = data['pid']
        camids = data['camid']
        if use_gpu:
            imgs = imgs.cuda()
        features = extractor(imgs)  # use the extractor instance directly
        features = features.cpu()
        f_.append(features)
        pids_.extend(pids.tolist())
        camids_.extend(camids.tolist())
    f_ = torch.cat(f_, 0)
    pids_ = np.asarray(pids_)
    camids_ = np.asarray(camids_)
    return f_, pids_, camids_

#Extracting features using extractor1
print('Extracting features from query set ...')
qf, q_pids, q_camids = feature_extraction(query_loader, extractor1, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(qf.size(0), qf.size(1)))

print('Extracting features from gallery set ...')
gf, g_pids, g_camids = feature_extraction(gallery_loader, extractor1, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(gf.size(0), gf.size(1)))



#Extracting features using extractor2
print('Extracting features from query set ...')
qf2, q_pids2, q_camids2 = feature_extraction(query_loader, extractor2, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(qf.size(0), qf.size(1)))

print('Extracting features from gallery set ...')
gf2, g_pids2, g_camids2 = feature_extraction(gallery_loader, extractor2, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(gf.size(0), gf.size(1)))


qf_avg = (qf + qf2  ) / 2
gf_avg = (gf + gf2 ) / 2

#Compute distance matrix for extractor1
print('Computing distance matrix with metric={} ...'.format("euclidean"))
distmat = metrics.compute_distance_matrix(qf_avg, gf_avg, metric="euclidean")
distmat = distmat.numpy()



print('Computing CMC and mAP ...')
cmc, mAP = metrics.evaluate_rank(
    distmat,
    q_pids,
    g_pids,
    q_camids,
    g_camids,
)

print("This is cmc",cmc)
# print results
print(f"mAP: {mAP:.1%}, Rank-1: {cmc[0]:.1%}, Rank-5: {cmc[4]:.1%}, Rank-10: {cmc[9]:.1%}, Rank-20: {cmc[19]:.1%}")

Model: densenet169
- params: 12,484,480
- flops: 2,193,096,704
Successfully loaded pretrained weights from "log/densenet169_ocreidfinal/model/model.pth.tar-100"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: densenet161
- params: 26,472,000
- flops: 5,045,354,496
Successfully loaded pretrained weights from "log/densenet161_ocreidfinal/model/model.pth.tar-100"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Extracting features from query set ...
Done, obtained 500-by-1664 matrix
Extracting features from gallery set ...
Done, obtained 500-by-1664 matrix
Extracting features from query set ...
Done, obtained 500-by-1664 matrix
Extracting features from gallery set ...
Done, obtained 500-by-1664 matrix


RuntimeError: The size of tensor a (1664) must match the size of tensor b (2208) at non-singleton dimension 1

## Training multiple models at a time

In [5]:
modelList = ["resnet50","resnet101","resnext50_32x4d","densenet121","densenet169", "densenet201","densenet161","osnet_x1_0","osnet_x0_75"] #"resnet50","resnet101","resnext50_32x4d","densenet121","densenet169", "densenet201","densenet161",

for modelName in modelList:
    #This is for Triplet loss
    datamanager = torchreid.data.ImageDataManager(
        root='reid-data',
        sources='occluded_reid',
        height=240,
        width=240,
        combineall=False,
        batch_size_train=20, #setting this at 64 crashes the run. IDK why lol
        batch_size_test=20,
        num_instances=4,
        workers=0,
        train_sampler='RandomIdentitySampler' # this is important
    )

    model = torchreid.models.build_model(
        name=modelName,
        num_classes=datamanager.num_train_pids,
        loss='triplet'
    )
    model = model.cuda()
    optimizer = torchreid.optim.build_optimizer(
        model, optim='adam', lr=0.0003,
    )
    scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=20
    )
    engine = torchreid.engine.ImageTripletEngine(
        datamanager, model, optimizer, margin=0.3,
        weight_t=0.7, weight_x=1, scheduler=scheduler
    )
    engine.run(
        max_epoch=100,
        save_dir=f'log/{modelName}_ocreidfinal',
        print_freq=10,
        normalize_feature= True
    )

Building train transforms ...
+ resize to 240x240
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 240x240
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded OccludedREID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   100 |      500 |         5
  query    |   100 |      500 |         5
  gallery  |   100 |      500 |         5
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded OccludedREID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   100 |      500 |         5
  query    |   100 |      500 |         5
  gallery  |   100 |      500 |         5
  ---------

In [3]:
modelList2 = ["densenet161"]#,"osnet_x1_0","osnet_x0_75" note: densenet161 is buggy as fuck when it comes to market1501. Ignore?
# done models "resnet50" ,"resnet101","densenet121","resnext50_32x4d","densenet169", "densenet201",
for modelName in modelList2:
    
    datamanager = torchreid.data.ImageDataManager(
        root="reid-data",
        sources="market1501",
        targets="market1501",
        height=256,
        width=128,
        batch_size_train=32,
        batch_size_test=100,
        transforms=["random_flip", "random_crop"]
    )

    model = torchreid.models.build_model(
        name=modelName,
        num_classes=datamanager.num_train_pids,
        loss='triplet'
    )
    model = model.cuda()
    optimizer = torchreid.optim.build_optimizer(
        model, optim='adam', lr=0.0003
    )
    scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=20
    )
    engine = torchreid.engine.ImageTripletEngine(
        datamanager, model, optimizer, margin=0.3,
        weight_t=0.7, weight_x=1, scheduler=scheduler
    )
    engine.run(
        max_epoch=100,
        save_dir=f'log/{modelName}_market1501final',
        print_freq=10,
        normalize_feature= True
    )


# modelList = ["resnet50","resnet101","resnext50_32x4d","densenet121","densenet169", "densenet201","densenet161","osnet_x1_0","osnet_x0_75"] #"resnet50","resnet101","resnext50_32x4d","densenet121","densenet169", "densenet201","densenet161",

# for modelName in modelList:
#     #This is for Triplet loss
#     datamanager = torchreid.data.ImageDataManager(
#         root='reid-data',
#         sources='occluded_reid',
#         height=240,
#         width=240,
#         combineall=False,
#         batch_size_train=20, #setting this at 64 crashes the run. IDK why lol
#         batch_size_test=20,
#         num_instances=4,
#         workers=0,
#         train_sampler='RandomIdentitySampler' # this is important
#     )

#     model = torchreid.models.build_model(
#         name=modelName,
#         num_classes=datamanager.num_train_pids,
#         loss='triplet'
#     )
#     model = model.cuda()
#     optimizer = torchreid.optim.build_optimizer(
#         model, optim='adam', lr=0.0003,
#     )
#     scheduler = torchreid.optim.build_lr_scheduler(
#         optimizer,
#         lr_scheduler='single_step',
#         stepsize=20
#     )
#     engine = torchreid.engine.ImageTripletEngine(
#         datamanager, model, optimizer, margin=0.3,
#         weight_t=0.7, weight_x=1, scheduler=scheduler
#     )
#     engine.run(
#         max_epoch=100,
#         save_dir=f'log/{modelName}_ocreidfinal',
#         print_freq=10,
#         normalize_feature= True
#     )

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset


c:\Users\User\Desktop\fyp project kaiyang\deep-person-reid\torchreid\data\datasets\image\market1501.py:37: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  warnings.warn(


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************


=> Start training
epoch: [1/100][10/404]	time 0.335 (1.602)	dat

KeyboardInterrupt: 

In [42]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    height=256,
    width=128,
    combineall=False,
    batch_size_test=32,
    batch_size_train=32,
    num_instances=4,
    train_sampler='RandomIdentitySampler' # this is important
)
model = torchreid.models.build_model(
    name='osnet_x0_75',
    num_classes=datamanager.num_train_pids,
    loss='triplet'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.ImageTripletEngine(
    datamanager, model, optimizer, margin=0.3,
    weight_t=0.7, weight_x=1, scheduler=scheduler
)
engine.run(
    max_epoch=100,
    save_dir='log/osnet_x0_75_market1501final',
    print_freq=10
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset


c:\Users\User\Desktop\fyp project kaiyang\deep-person-reid\torchreid\data\datasets\image\market1501.py:37: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  warnings.warn(


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




### Loading the models to check their results for Market-1501

In [20]:
weight_path = "log/resnext50_32x4d_market1501final/model/model.pth.tar-100"
model = torchreid.models.build_model(
    name='resnext50_32x4d',
    num_classes=datamanager.num_train_pids,
    loss='triplet'
)

model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.ImageTripletEngine(
    datamanager, model, optimizer, margin=0.3,
    weight_t=0.7, weight_x=1, scheduler=scheduler
)

torchreid.utils.load_pretrained_weights(model, weight_path)
engine.run(
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=True,
)

Successfully loaded pretrained weights from "log/resnext50_32x4d_market1501final/model/model.pth.tar-100"
##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3368-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-2048 matrix
Speed: 0.0089 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 67.9%
CMC curve
Rank-1  : 84.8%
Rank-5  : 93.4%
Rank-10 : 95.7%
Rank-20 : 97.6%


### Loading the models to check their result Occluded-Reid

In [31]:
weight_path = "log/osnet_x0_75_ocreidfinal/model/model.pth.tar-100"

model = torchreid.models.build_model(
    name='osnet_x0_75',
    num_classes=datamanager.num_train_pids,
    loss='triplet'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003,
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.ImageTripletEngine(
    datamanager, model, optimizer, margin=0.3,
    weight_t=0.7, weight_x=1, scheduler=scheduler
)

torchreid.utils.load_pretrained_weights(model, weight_path)
engine.run(
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=True,
)


Successfully loaded imagenet pretrained weights from "C:\Users\User/.cache\torch\checkpoints\osnet_x0_75_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Successfully loaded pretrained weights from "log/osnet_x0_75_ocreidfinal/model/model.pth.tar-100"
##### Evaluating occluded_reid (source) #####
Extracting features from query set ...
Done, obtained 500-by-512 matrix
Extracting features from gallery set ...
Done, obtained 500-by-512 matrix
Speed: 0.0197 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 46.1%
CMC curve
Rank-1  : 53.8%
Rank-5  : 75.2%
Rank-10 : 83.4%
Rank-20 : 89.6%
